In [ ]:
import numpy as np
import robotic as ry
from tqdm import tqdm
import matplotlib.pyplot as plt
from Robotic_Manipulation.shelf import generate_target_box

In [ ]:
# Define the new config
C = ry.Config()
C.addFile(ry.raiPath("scenarios/pandaSingle.g"))

generate_target_box(C, [-.3, .3, .75])

box_frame = C.addFrame("box") \
    .setPosition([.3, .3, .8]) \
    .setShape(ry.ST.ssBox, size=[.05, .05, .12, .001]) \
    .setColor([1., 1., 0.]) \
    .setContact(1) \
    .setMass(1.)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

bot = ry.BotOp(C, False)
bot.home(C)
bot.gripperMove(ry._left)
while not bot.gripperDone(ry._left):
    bot.sync(C)
qHome = C.getJointState()

In [ ]:
autoencoder = Autoencoder(latent_dim=LATENT_DIM).to(device)
autoencoder.load_state_dict(torch.load('./models/autoencoder_sim.pth', map_location=device))

with open('./models/ridge_model.pkl', 'rb') as file:
    ridge_model = pickle.load(file)

In [ ]:
for i in range(100):
    # Get input data
    frame, depth = bot.getImageAndDepth("cameraTop")
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (IMAGE_RESIZE, IMAGE_RESIZE))
    image = image/255.0
    image = np.transpose(image, (2, 0, 1))  # HWC to CHW

    # Perform inference
    latent_im = autoencoder.encode(torch.Tensor([image]).to(device))
    latent_im = latent_im.detach().cpu().numpy()
    waypoints = ridge_model.predict(latent_im)[0].reshape(-1, 3)
    print(waypoints.shape)
    for i, way in enumerate(waypoints):
        frame = C.getFrame(f"way{i}")
        if frame:
            frame.setPosition(way)
            print(way)
        else:
            frame = C.addFrame(f"way{i}").setShape(ry.ST.marker, [.2]).setPosition(way)
        if i == 0:
            frame.setColor([1., 0., 0., 1.])
    C.view()
    del bot
    x_dev = np.random.random() * .4 -.2
    y_dev = np.random.random() * .3 -.15
    box_frame.setQuaternion([1., 0., 0., 0.])
    box_frame.setPosition([.3 + x_dev, .3 + y_dev, .8])
    C.setJointState(qHome)
    C.view()
    bot = ry.BotOp(C, useRealRobot=False)
    bot.home(C)
    bot.gripperMove(ry._left)
    bot.sync(C, 2.)

In [ ]:
# Go through waypoints
komo = ry.KOMO(C, waypoints.shape[0], 20, 2, False)
komo.addControlObjective([], 1, .1)
komo.addControlObjective([], 2, .01)

for i, way in enumerate(waypoints):
    komo.addObjective([i+1.], ry.FS.position, ["l_gripper"], ry.OT.eq, [1e1], way)

komo.addObjective([1., waypoints.shape[0]], ry.FS.vectorZ, ["l_gripper"], ry.OT.eq, [1e1], [0, 0, 1])

sol = ry.NLP_Solver()
sol.setProblem(komo.nlp())
sol.setOptions(damping=1e-3, verbose=0, stopTolerance=1e-3, maxLambda=100., stopEvals=200)
ret = sol.solve()

if ret.feasible:
    print("Feasible")
    bot.move(komo.getPath(), [10.])
    bot.wait(C)
    bot.home(C)
else:
    print("Path infeasible!")